In [ ]:
# домашняя работа 

In [ ]:
# 1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. 
#    Использовать датасет 
#    https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
# 2. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию 
#    для оценки качества моделей
# 3. Построить случайный лес, вывести важность признаков
# 4. Обучить стекинг как минимум 3х моделей, использовать 
#    хотя бы 1 линейную модель и 1 нелинейную
# 5. Для валидации модели 2-го уровня использовать 
#    отдельный hold-out датасет, как на занятии
# 6. Показать, что использование ансамблей моделей действительно 
#    улучшает качество (стекинг vs другие модели сравнивать на hold-out)
# 7. В качестве решения: Jupyter notebook с кодом, комментариями 
#    и графиками, ссылка на гитхаб


In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [17]:
data = pd.read_csv('train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [19]:
# Заполнить средними значениеми
def put_mean_to_na_param(dataset, list_columns):
    for i in list_columns:
        dataset[i].fillna(dataset[i].mean(), inplace=True)

In [20]:
# Заполнить значением
def put_value_to_na_param(dataset, list_columns, value=0 ):
    for i in list_columns:
        dataset[i].fillna(value, inplace=True)

In [21]:
# Заполнить самым распространенным значением
def put_most_common_meaning_to_na_param(dataset, list_columns):
    for i in list_columns:
        dataset[i].fillna(dataset[i].value_counts()[0], inplace=True)

In [22]:
# Заполнить значения NaN значениеями из другой колонки
def put_param_from_col_src_to_na_param(dataset, col_dest, col_src): 
    dataset[col_dest]=dataset.apply(lambda x: x[col_src] if x[col_dest] is not None else x[col_dest], axis=1 )
            

In [23]:
# Закодировать nominal категориальную переменную с помощью "one hot encoding"  (используем реализацию из лекций)
def get_one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df 

# Закодировать ordered категориальную переменную используя числовой порядок
def get_ordered_col(dataset, col, dict_col_params):
    """
    В функцию передается датасет, имя колонки и словарь {'значение_параметра_в_колонке': числовое_значение_для_замены }
    """
    dataset[col + '_order'] = dataset.apply(lambda x: dict_col_params[x[col]],axis=1)
    return dataset

def get_ordered_cols(dataset, cols, dict_col_params):
    for col in cols:
        res = get_ordered_col(dataset, col, dict_col_params)
    return res.drop(columns=cols)     

dict_label_5_1 = {'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'Po': 0}
dict_label_5_2 = {'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'NA': 0}
dict_label_5_3 = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0} 
dict_label_6_1 = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
dict_label_7_1 = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0} 


In [24]:
# функция подготовки данных
def get_data(dataset):
    put_value_to_na_param(dataset, ['MiscFeature','Alley','Fence','BsmtQual','PoolQC','BsmtCond',\
                                    'BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu',\
                                    'GarageType','GarageFinish','GarageQual','GarageCond'], 'NA' )
    
    put_value_to_na_param(dataset, ['MasVnrArea'], 0)
    put_value_to_na_param(dataset, ['MasVnrType'], 'none')                      
    put_mean_to_na_param(dataset, ['LotFrontage'])
    put_most_common_meaning_to_na_param(dataset, ['Electrical'])   
    put_param_from_col_src_to_na_param(dataset, 'GarageYrBlt', 'YearRemodAdd')     
    
    data_raw = dataset.copy()
    data_raw = get_one_hot(dataset, ['MiscFeature', 'Fence', 'Alley', 'Street', 'LotShape', 'LandContour', 'Utilities',\
                                     'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', 'Foundation', 'Heating', \
                                     'CentralAir', 'Electrical', 'PavedDrive', 'MSZoning','MSSubClass','Neighborhood','Condition1',\
                                     'Condition2','HouseStyle','RoofStyle','RoofMatl','Exterior1st',\
                                     'Exterior2nd','Functional','GarageType','GarageFinish',\
                                     'SaleType','SaleCondition'])
    
    data_raw = get_ordered_cols(data_raw, ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual'], dict_label_5_1)
    data_raw = get_ordered_cols(data_raw, ['PoolQC'], dict_label_5_2)
    data_raw = get_ordered_cols(data_raw, ['BsmtExposure'], dict_label_5_3)
    data_raw = get_ordered_cols(data_raw, ['BsmtQual', 'BsmtCond', 'FireplaceQu', 'GarageQual', 'GarageCond'] , dict_label_6_1)
    data_raw = get_ordered_cols(data_raw, ['BsmtFinType1', 'BsmtFinType2'] , dict_label_7_1) 
    
    y = data_raw['SalePrice'].copy()
    #data_raw= data_raw.drop(columns=['SaleCategory'])
    columns_to_drop = ['SalePrice', 'MSZoning','MSSubClass','Neighborhood','Condition1',\
                       'Condition2','HouseStyle','RoofStyle','RoofMatl','Exterior1st',\
                       'Exterior2nd','Functional','GarageType','GarageFinish',\
                       'SaleType','SaleCondition','MiscFeature', 'Fence', 'Alley',\
                       'Street', 'LotShape', 'LandContour', 'Utilities',\
                       'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', \
                       'Foundation', 'Heating', 'CentralAir', 'Electrical', 'PavedDrive']
    
    data_raw = data_raw.drop(columns=columns_to_drop)
    
        
    
    return data_raw, y
    

In [25]:
X_full, y_full = get_data(data)

In [26]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 263 entries, Id to BsmtFinType2_order
dtypes: float64(2), int64(47), uint8(214)
memory usage: 864.1 KB


In [27]:
X_full.head()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,KitchenQual_order,PoolQC_order,BsmtExposure_order,BsmtQual_order,BsmtCond_order,FireplaceQu_order,GarageQual_order,GarageCond_order,BsmtFinType1_order,BsmtFinType2_order
0,1,65.0,8450,7,5,2003,2003,196.0,706,0,...,3,0,1,4,3,0,3,3,6,1
1,2,80.0,9600,6,8,1976,1976,0.0,978,0,...,2,0,4,4,3,3,3,3,5,1
2,3,68.0,11250,7,5,2001,2002,162.0,486,0,...,3,0,2,4,3,3,3,3,6,1
3,4,60.0,9550,7,5,1915,1970,0.0,216,0,...,3,0,1,3,4,4,3,3,5,1
4,5,84.0,14260,8,5,2000,2000,350.0,655,0,...,3,0,3,4,3,3,3,3,6,1


In [28]:
# X и y общая выборка для моделей 1 уровня 
# X_out,y_out - выборка для проверки модели второго уровня

X, X_out, y, y_out = train_test_split(X_full.drop(columns=['Id']), y_full, test_size=0.1 ,random_state=42)

In [29]:
# 3. Построить случайный лес, вывести важность признаков

from sklearn.ensemble import RandomForestClassifier

In [30]:
clf = RandomForestClassifier()
clf.fit(X,y)

feature_importances= {}
for col, val in zip(X.columns, clf.feature_importances_):
    feature_importances[col] = val
    
sorted_x = sorted(feature_importances.items(), key=lambda kv: kv[1], reverse=True)
sorted_x[:20]

[('GrLivArea', 0.03210102325943861),
 ('GarageArea', 0.030026470006037986),
 ('TotalBsmtSF', 0.02737552048670964),
 ('YearBuilt', 0.027245004907455565),
 ('BsmtUnfSF', 0.027200269816432244),
 ('BsmtFinSF1', 0.02629225865667014),
 ('1stFlrSF', 0.02626189702722804),
 ('LotArea', 0.026214931961214327),
 ('MoSold', 0.024344744671230666),
 ('YearRemodAdd', 0.023722979525410533),
 ('GarageYrBlt', 0.022583184997811097),
 ('LotFrontage', 0.021649133436188288),
 ('YrSold', 0.02006803332412493),
 ('OpenPorchSF', 0.01888221560827332),
 ('WoodDeckSF', 0.017952927440563555),
 ('MasVnrArea', 0.01722465414872774),
 ('TotRmsAbvGrd', 0.015974339776946213),
 ('2ndFlrSF', 0.014340974116049782),
 ('BsmtFinType1_order', 0.013983180345690275),
 ('OverallQual', 0.013545274205623948)]

In [31]:
# 4. Обучить стекинг как минимум 3х моделей, использовать 
#    хотя бы 1 линейную модель и 1 нелинейную
# 5. Для валидации модели 2-го уровня использовать 
#    отдельный hold-out датасет, как на занятии

In [32]:
# Для стекинга используем следующие типы моделей
# модели первого уровня
# 1. SGDRegressor
# 2. Ridge
# 3. KNeighborsRegressor
# модель второго уровня
# 1. LinearRegression

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [75]:
def make_stacking(X, y, dict_model, meta_model):
    kf = KFold(n_splits=len(dict_model.keys()))
    result_fold = {}
    i = 0
    for train_index, test_index in kf.split(X):
        k = list(dict_model.keys())[i]
        result_fold[k] = {'X_train': X.iloc[train_index], 'X_test': X.iloc[test_index],
                          'y_train': y.iloc[train_index], 'y_test': y.iloc[test_index]}
        i+=1
    result_model ={}
    for j in dict_model.keys():
        print(result_fold[j]['X_train'].shape)
        print(result_fold[j]['y_train'].shape)
        dict_model[j].fit(result_fold[j]['X_train'], result_fold[j]['y_train'])
        result_model[j] = dict_model[j].predict(result_fold[j]['X_test'])
    df_result = pd.DataFrame.from_dict(result_model)
    meta_model.fit(df_result,result_fold[j]['y_test'])
    return dict_model, meta_model

In [52]:
def make_blanding(X, y, dict_model, meta_model):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)
    result_model ={}
    for i in dict_model.keys():
        dict_model[i].fit(X_train, y_train)
        result_model[i] = dict_model[i].predict(X_test)
    df_result = pd.DataFrame.from_dict(result_model)
    meta_model.fit(df_result,y_test)
    return dict_model, meta_model

In [47]:
def predict_result(X, dict_model, meta_model):
    result_model ={}
    for i in dict_model.keys():
        result_model[i] = dict_model[i].predict(X)
    df_result = pd.DataFrame.from_dict(result_model)
    result = meta_model.predict(df_result) 
    return result

In [77]:
dict_model = {}
dict_model['LinearRegression'] = LinearRegression()
dict_model['Ridge'] = Ridge()
dict_model['SGDRegressor'] = SGDRegressor()

meta_model = KNeighborsRegressor()

blending_dict_model, blending_meta_model = make_blanding(X, y, dict_model, meta_model)
stacking_dict_model, stacking_meta_model = make_stacking(X, y, dict_model, meta_model)

blending_result = predict_result(X_out, blending_dict_model, blending_meta_model)
stacking_result = predict_result(X_out, stacking_dict_model, stacking_meta_model)


(876, 262)
(876,)
(876, 262)
(876,)
(876, 262)
(876,)


/home/user/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [80]:
from sklearn.metrics import mean_squared_error

In [81]:
print('blending:', mean_squared_error(blending_result,y_out))
print('stacking:', mean_squared_error(stacking_result,y_out))

blending: 3546119623.6498632
stacking: 3546119623.6498632


In [83]:
from sklearn.metrics import r2_score

In [84]:
print('blending:', r2_score(blending_result,y_out))
print('stacking:', r2_score(stacking_result,y_out))

blending: 0.06503459453022209
stacking: 0.06503459453022209


In [59]:
d = pd.DataFrame.from_dict({1:[1,2,3],2:[2,3,4],3:[3,4,5]})

In [68]:
d.iloc[[1,2]]

,1,2,3
1,2,3,4
2,3,4,5
